In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import glob
import os

In [2]:
to_idr_df = pd.read_csv("comodity-price-prediction-penyisihan-arkavidia-9/to_idr.csv")
to_idr_df

,Date,USD/IDR,MYR/IDR
0,2022-01-03,14215.000000,3404.790334
1,2022-01-04,14283.599609,3425.323486
2,2022-01-05,14385.500000,3438.216923
3,2022-01-06,14446.500000,3446.207084
4,2022-01-07,14408.000000,3421.515007
...,...,...,...
711,2024-09-24,15174.099609,3612.450806
712,2024-09-25,15121.099609,3643.638324
713,2024-09-26,15201.500000,3682.087841
714,2024-09-27,15070.000000,3640.096724


* CPO Price in MYR
* US Sugar 11 Price in USD
* US Wheat in USD

In [3]:
all_files = glob.glob("comodity-price-prediction-penyisihan-arkavidia-9/Global Commodity Price/" + "*.csv")
print(all_files)

['comodity-price-prediction-penyisihan-arkavidia-9/Global Commodity Price/US Sugar 11 Futures Historical Data.csv', 'comodity-price-prediction-penyisihan-arkavidia-9/Global Commodity Price/Natural Gas Futures Historical Data.csv', 'comodity-price-prediction-penyisihan-arkavidia-9/Global Commodity Price/US Wheat Futures Historical Data.csv', 'comodity-price-prediction-penyisihan-arkavidia-9/Global Commodity Price/Crude Oil WTI Futures Historical Data.csv', 'comodity-price-prediction-penyisihan-arkavidia-9/Global Commodity Price/Palm Oil Futures Historical Data.csv', 'comodity-price-prediction-penyisihan-arkavidia-9/Global Commodity Price/Newcastle Coal Futures Historical Data.csv']


In [4]:
lst = []

for file in all_files:
    df = pd.read_csv(file)

    commodity_name = os.path.basename(os.path.splitext(file)[0])
    df["Price"] = df["Price"].astype(str).str.replace(",", "").astype(float)

    
    if commodity_name == "Palm Oil Futures Historical Data" or commodity_name == "US Wheat Futures Historical Data" or commodity_name == "US Sugar 11 Futures Historical Data":
        # print(df[["Date", "Price", "Change %"]])
        if commodity_name == "Palm Oil Futures Historical Data":
            df["Price"] = df["Price"] * to_idr_df["MYR/IDR"]
        else:
            df["Price"] = df["Price"] * to_idr_df["USD/IDR"]

        df["Change %"] = df["Change %"].str.rstrip("%").astype(float) / 100

        df = df.rename(columns={"Price": f"Price_{commodity_name}", "Change %": f"Change_{commodity_name}"})
    
        lst.append(df[["Date", f"Price_{commodity_name}", f"Change_{commodity_name}"]])


merged_df = pd.DataFrame({"Date": []})

for df in lst:
    merged_df = pd.merge(right=merged_df, left=df, how="outer", on="Date")



In [5]:
merged_df.head()

,Date,Price_Palm Oil Futures Historical Data,Change_Palm Oil Futures Historical Data,Price_US Wheat Futures Historical Data,Change_US Wheat Futures Historical Data,Price_US Sugar 11 Futures Historical Data,Change_US Sugar 11 Futures Historical Data
0,2022-01-03,1.843590e+07,0.0194,1.170686e+07,-0.0165,290170.16,-0.0074
1,2022-01-04,1.846547e+07,0.0093,1.196033e+07,0.0158,289537.50,0.0005
2,2022-01-05,1.885803e+07,0.0243,1.180692e+07,-0.0120,284691.82,-0.0219
3,2022-01-06,1.865143e+07,-0.0092,1.158165e+07,-0.0194,285273.77,-0.0082
4,2022-01-07,1.843714e+07,-0.0106,1.171488e+07,0.0168,284377.75,-0.0077


In [6]:
merged_df.to_csv("comodity-price-prediction-penyisihan-arkavidia-9/global_commodity_price_merged.csv", index=False)